In [ ]:
import pandas as pd
import numpy as np
import os

# Caminhos
INPUT_PATH = "../data/01_raw/dataset_kobe_prod.parquet"
OUTPUT_PATH = "../data/04_feature/data_features_prod.parquet"

In [ ]:
from gettext import Catalog


raw_data = Catalog.load("data_shots")

In [4]:
df = pd.read_parquet(INPUT_PATH)
print("\n📊 Informações gerais:")
print(df.info())
print("\n📈 Estatísticas descritivas:")
print(df.describe())


📊 Informações gerais:
<class 'pandas.core.frame.DataFrame'>
Index: 5412 entries, 10 to 30695
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lat                5412 non-null   float64
 1   lon                5412 non-null   float64
 2   minutes_remaining  5412 non-null   int64  
 3   period             5412 non-null   int64  
 4   playoffs           5412 non-null   int64  
 5   shot_distance      5412 non-null   int64  
 6   shot_made_flag     5412 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 338.2 KB
None

📈 Estatísticas descritivas:
               lat          lon  minutes_remaining       period     playoffs  \
count  5412.000000  5412.000000        5412.000000  5412.000000  5412.000000   
mean     33.849585  -118.263931           4.087029     2.703806     0.136918   
std       0.084045     0.157584           3.442213     1.146149     0.343792   
min      33.253300  -118.519800     

In [ ]:
# ===== FEATURE ENGINEERING =====

# Exemplo 1: Criar uma coluna de distância ao cesto (hipotética)
if {"loc_x", "loc_y"}.issubset(df.columns):
    df["distance_to_hoop"] = np.sqrt(df["loc_x"] ** 2 + df["loc_y"] ** 2)

# Exemplo 2: Criar uma coluna para saber se foi um arremesso de 3 pontos
if "shot_type" in df.columns:
    df["is_three_point"] = df["shot_type"].apply(lambda x: 1 if "3PT" in x else 0)

# Exemplo 3: Codificar variáveis categóricas com poucas categorias
low_card_cat_cols = [
    col for col in df.select_dtypes(include="object") if df[col].nunique() <= 10
]
df = pd.get_dummies(df, columns=low_card_cat_cols, drop_first=True)

# Exemplo 4: Preencher valores ausentes
df.fillna(df.median(numeric_only=True), inplace=True)

# Verificar novamente os nulos
print("\n✅ Após tratamento, valores ausentes:")
print(df.isnull().sum().sum())

# Separar X e y, se a variável target existir
if "shot_made_flag" in df.columns:
    df = df.dropna(subset=["shot_made_flag"])  # Remove linhas sem target
    print(f"\n🎯 Target presente: {df['shot_made_flag'].value_counts(normalize=True)}")

# Salvando os dados processados
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
df.to_parquet(OUTPUT_PATH, index=False)
print(f"\n💾 Dados salvos com sucesso em: {OUTPUT_PATH}")


🔍 Valores ausentes por coluna:
lat                  0
lon                  0
minutes_remaining    0
period               0
playoffs             0
shot_distance        0
shot_made_flag       0
dtype: int64
